# 1. 讀取檔案

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# 使用 pandas 的 read_csv 函式讀取訓練資料
df = pd.read_csv("data/train.csv")

# 2.資料預處理

In [12]:
# A. 處理 'horsepower' (馬力)
# 強制轉為數字，無法轉的變 NaN，然後用中位數補
df['horsepower'] = pd.to_numeric(df['horsepower'], errors='coerce')
df['horsepower'] = df['horsepower'].fillna(df['horsepower'].median())

# B. 處理 'origin' (產地) - 修正 'usa' 報錯問題
def clean_origin(x):
    # 轉字串 -> 轉小寫 -> 去除前後空白 (這步最關鍵)
    s = str(x).lower().strip()
    
    # 判斷各種可能的寫法
    if s in ['usa', 'united states', 'america', '1', '1.0']:
        return 1
    elif s in ['europe', '2', '2.0']:
        return 2
    elif s in ['japan', '3', '3.0']:
        return 3
    else:
        # 遇到無法辨識的資料 (如亂碼)，預設歸類為 1 (美國)
        return 1

# 套用清洗函數
df['origin'] = df['origin'].apply(clean_origin)

# [檢查點] 確保 origin 欄位真的是數字，不是物件
# 如果這裡印出 object，代表還有字串沒清乾淨
print("Origin 欄位唯一值檢查 (應全為數字):", df['origin'].unique())

# C. 移除其他缺失值
df_clean = df.dropna()
print(f"預處理完成。清洗後形狀: {df_clean.shape}")

Origin 欄位唯一值檢查 (應全為數字): [1 3 2]
預處理完成。清洗後形狀: (318, 10)


# 3.特徵工程

In [13]:
# --- 3. 特徵工程 ---
features = ['weight', 'acceleration', 'model_year', 'cylinders', 'displacement', 'horsepower', 'origin']
target = 'mpg'

X = df_clean[features]
y = df_clean[target]


# 4. 訓練模型

In [ ]:
# --- 4. 訓練模型 ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
print("開始訓練模型...")
model.fit(X_train, y_train)  
print("模型訓練成功！")

# 預測與評估
train_predictions = model.predict(X_train)
test_predictions = model.predict(X_test)

train_rmse = np.sqrt(mean_squared_error(y_train, train_predictions))
test_rmse = np.sqrt(mean_squared_error(y_test, test_predictions))

print(f"\n訓練誤差 (Train RMSE): {train_rmse:.4f}")
print(f"測試誤差 (Test RMSE):  {test_rmse:.4f}")

開始訓練模型...
模型訓練成功！

訓練誤差 (Train RMSE): 3.3858
測試誤差 (Test RMSE):  3.5723


# 5.輸出提交檔案

In [17]:
# --- 5. 輸出提交檔案 ---
# 讀取測試檔案
df_test = pd.read_csv("data/test.csv")
# 對測試集做一模一樣的處理
df_test['horsepower'] = pd.to_numeric(df_test['horsepower'], errors='coerce')
df_test['horsepower'] = df_test['horsepower'].fillna(df['horsepower'].median()) # 注意：實務上最好用訓練集的中位數
df_test['origin'] = df_test['origin'].apply(clean_origin) # 同樣套用清洗函數
df_test = df_test.fillna(0)

# 預測
predictions = model.predict(df_test[features])

# 建立提交檔
submission_df = pd.DataFrame({'id': df_test['id'], 'mpg': predictions})
submission_df.to_csv("submission.csv", index=False)
print("\n檔案 submission.csv 已生成！")
submission_df.to_csv('submission.csv', index=False)
print("提交文件 'submission.csv' 已成功生成！")


檔案 submission.csv 已生成！
提交文件 'submission.csv' 已成功生成！


# 6. 報告

姓名：____葉治廷______ 學號：__110401062________

第一部分：準確度分數 (Accuracy Scores) (1分) 我的準確度分數： 3.5723

第二部分：我的實驗記錄 (My Experiment Log) (3分)

【實驗 1】

我做的修改： 針對 horsepower 欄位處理。將 "?" 亂碼轉為 NaN 並使用中位數填補，取代原本刪除整行的做法。

結果與觀察 (分數變化、心得等)： (同前文) 雖然解決了馬力資料缺失的問題，大幅提升了模型可用性，但此時尚未處理產地 (origin) 的格式錯誤，因此還沒加入產地特徵，預測能力還有進步空間。

該次實驗分數： 3.8450 
【實驗 2】

我做的修改： 修正 origin (產地) 格式並加入訓練特徵。撰寫函式將 "usa" 等文字轉為數字編碼。

結果與觀察 (分數變化、心得等)： (同前文) 加入產地資訊後，模型能區分不同國家車輛的油耗特性（如日本車較省油）。觀察到 RMSE 從 3.8 左右下降到 3.57，證明這個特徵對預測很有幫助。

該次實驗分數： 3.5723 

第三部分：總結與心得 (維持原樣即可) 內容：(1) 我認為最有效的是 horsepower 的清洗與填補，因為它是物理上影響油耗最大的因素，救回這些資料比什麼都重要。(2) 學習到資料預處理中 fit 前檢查資料型態的重要性，解決了字串轉浮點數的報錯問題。